# Convolutional Neural Networks

In this tutorial we continue our journey in deep learning by looking at convolutional neural networks or ConvNets. We start by re viewing how convolution work. Then we try to solve a steel microstructure classification problem by using a CNN. We explore how data augmentation, regularization and other gradient descent algorithms may help to better train the model.

Run the notebook in Google colab:
https://colab.research.google.com/github/heprom/cvml/blob/main/corrections/CNN_cor.ipynb

You should use GPU acceleration to train your network, on google colab, before starting working, go to Execution -> Modifier le Type d'Execution -> select GPU as hardware accelerator

In [ ]:
import numpy as np
import matplotlib
%matplotlib inline
from matplotlib import pyplot as plt, cm

## Understanding convolutions

In thisfirst section, we experiment convolutions on images using simple `numpy` operations. We first work with a single channel image from MNIST and then a 3 channel RGB image of a cat.

In [ ]:
 sklearn.datasets import load_digits
mnist = load_digits()

create a variable `image` to hold the first $8\times8$ image in the data set

In [ ]:
image = ...
print(image)
plt.imshow(..., cmap=cm.gray_r)
plt.axis('off')
plt.show()

create the following $3\times 3$ kernel as a numpy array: $\left[\begin{array}{ccc}-1 & 0 & +1 \\ -2 & 0 & +2 \\ -1 & 0 & +1\end{array}\right]$

In [ ]:
kernel = np.array(...)
print(kernel)

pad the image with zeros.

In [ ]:
kernel_size = ...
pad = ...
im = np.pad(image, ((pad, pad), (pad, pad)), mode='constant')
print('paded image size is now {}'.format(im.shape))

convolve the kernel with the image. Create an algorithm using for loops to output the convolution to a new variable `conv`.

In [ ]:
conv = np.empty_like(image)

for i in range(pad, im.shape[0] - pad):
    for j in range(pad, im.shape[1] - pad):
        subset = im[i-pad:i+pad+1, j-pad:j+pad+1]
        conv[i-pad, j-pad] = ...  # element-wise multiplication

print('output size of the convolution is {}'.format(conv.shape))
plt.imshow(conv, cmap=cm.gray)
plt.axis('off')
plt.show()

now this works, make a function called `convolve` which takes for input an image, a kernel and output the result of the convolution. Assume image is in form (n x m x channels) and represented by floats in the [0, 1] range.

In [ ]:
def convolve(image, kernel):
    kernel_size = ...
    pad = ...
    im = np.pad(image, ((pad, pad), (pad, pad)), mode='constant')
    conv = np.empty_like(image)

    for i in range(pad, im.shape[0] - pad):
        for j in range(pad, im.shape[1] - pad):
            # get the (i, j) subset of size (2 x pad + 1)
            subset = im[i - pad:i + pad + 1, j - pad:j + pad + 1]
            # perform the convolution
            conv[i - pad, j - pad] = ...
    conv = (conv - conv.min()) / (conv.max() - conv.min())  # normalization
    return conv

In [ ]:
plt.imshow(convolve(image, kernel), cmap=cm.gray)
plt.axis('off')
plt.show()

Now let's work with a 3 channel RGB image. Load it, convert if to float representation and in gray scale mode.

In [ ]:
from skimage import data
cat = data.chelsea().astype(np.float)
cat = ...  # convert to gray scale
cat /= cat.max()  # with float representation, the range is [0, 1]
print(cat.shape)
print(cat.dtype)
print(cat.max())

In [ ]:
plt.imshow(cat, cmap=cm.gray)
plt.axis('off')
plt.show()

create all the following kernel and try them out:

 - Blur kernel: $\left[\begin{array}{ccc}1 & 1 & 1 \\ 1 & 1 & 1 \\ 1 & 1 & 1\end{array}\right]$
 - Laplacian kernel: $\left[\begin{array}{ccc}0 & 1 & 0 \\ 1 & -4 & 1 \\ 0 & 1 & 0\end{array}\right]$
 - Emboss kernel: $\left[\begin{array}{ccc}-2 & -1 & 0 \\ -1 & 1 & 1 \\ 0 & 1 & 2\end{array}\right]$

In [ ]:
# blur filters
blur3 = ...
blur5 = ...
blur7 = ...

# sharpen
sharpen = ...

# Laplacian kernel
laplacian = ...

# construct the Sobel x-axis kernel
sobelX = np.array(([-1, 0, 1], [-2, 0, 2], [-1, 0, 1]))

# construct the Sobel y-axis kernel
sobelY = np.array(([-1, -2, -1], [0, 0, 0], [1, 2, 1]))

# construct an emboss kernel
emboss = ...

kernels = [blur3, sharpen, laplacian, emboss]
kernel_labels = ['blur3', 'sharpen', 'laplacian', 'emboss']

In [ ]:
fig = plt.figure(figsize=(10, 7))
for i in range(len(kernels)):
    ax = plt.subplot(2, 2, i + 1)
    convolution = ...
    plt.imshow(convolution, cmap=cm.gray, vmin=0.6, vmax=0.9)
    plt.title(kernel_labels[i])
    plt.axis('off')
plt.show()

## Automated classification of material microstructure : our First ConvNet

This model adopts a very classical architecture, so it is perfect to get started with CNN. It can be summarized as:

```INPUT => 3 x (CONV => RELU => POOLING) => FC(128) => FC(3)```

### As usual start by loading our data set

Here we will work with microstructure images with 3 classes: A, B and C. A data set micrographs_64.npz has been created with all the microstructure images and the labels.

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf

with np.load(...) as data:
  X = data['X']
  y = data['y']
  labels = data['labels']

# observe that each image is 3D with one channel (gray level images)
print('data set contains %d images of size %d x %d x %d' % (X.shape[0], X.shape[1], X.shape[2], X.shape[3]))

# partition the data into training and testing, use 20% for the test set
(X_train, X_test, y_train, y_test) = ...

# we use the new tensorflow Dataset functionalities 
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))

BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 32

train_ds = train_ds.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_ds = test_ds.batch(BATCH_SIZE)

Show some of the images with the labels to have an idea of how the data looks

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 7))
for images, image_labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy())
    plt.title(labels[image_labels[i]])
    plt.axis("off")

Partition the data into training and testing splits using 75% of the data for training and the remaining 25% for testing

In [ ]:
# display micrographs from a given class
class_A = X_train[np.where(y_train == 0)]
class_B = X_train[np.where(y_train == 1)]
class_C = X_train[np.where(y_train == 2)]
class_A.shape

N = 8
M = 4
indices = np.random.randint(0, class_B.shape[0], size=N*M)

plt.figure(figsize=(15, 8))
for i in range(N * M):
    ax = plt.subplot(M, N, i + 1)
    plt.imshow(class_B[indices[i]])
    plt.axis('off')
plt.show()

In [ ]:
# check the histogram for each class
ax1 = plt.subplot(1, 3, 1)
ax1.hist(..., bins=25)
plt.title('class A')
ax2 = plt.subplot(1, 3, 2)
ax2.hist(..., bins=25)
plt.title('class B')
ax3 = plt.subplot(1, 3, 3)
ax3.hist(..., bins=25)
plt.title('class C')
plt.show()

### Build the model with Keras

start importing useful stuff from Keras

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

num_classes = len(labels)

model = Sequential([
  layers.Conv2D(..., 3, padding='same', input_shape=..., activation=...),
  layers.MaxPooling2D(),
  layers.Conv2D(..., 3, padding='same', activation=...),
  layers.MaxPooling2D(),
  layers.Conv2D(..., 3, padding='same', activation=...),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(..., activation='relu'),
  layers.Dense(...)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
# train the model
epochs = 15
history = model.fit(
  train_ds,
  validation_data=test_ds,
  epochs=epochs
)

Save our model to the disk (to reuse it later), this is called **serialization**. In Keras, the architecture of the model and the trained weights are save to a HDF5 file.

In [ ]:
model.save('microstructure_cnn.keras')

Now evaluate the network

In [ ]:
predictions = model.predict(..., batch_size=32)

In [ ]:
# on train data
index = 5
plt.imshow(X_train[index])
plt.title('predicted as %c / gt is %c' % (labels[predictions[index].argmax()], labels[y_train[index].argmax()]))
plt.axis('off')
plt.show()

In [ ]:
predictions = model.predict(X_test, batch_size=32)

In [ ]:
# on test data
index = 23
plt.imshow(X_test[index])
plt.title('predicted as %c / gt is %c' % (labels[predictions[index].argmax()], labels[y_test[index].argmax()]))
plt.axis('off')
plt.show()

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, predictions.argmax(axis=1), target_names=labels))

In [ ]:
# plot the training loss and accuracy
plt.figure()
plt.plot(history.history["loss"], label="train_loss")
plt.plot(history.history["val_loss"], label="val_loss")
plt.plot(history.history["accuracy"], label="train_acc")
plt.plot(history.history["val_accuracy"], label="val_acc")
plt.title('Training Loss and Accuracy')
plt.xlabel('Epoch #')
plt.ylabel('Loss/Accuracy')
plt.legend()
plt.show()

## Using data augmentation and regularization

Finally for this tutorial we try this to use data augmentation since we may not have so many images and we start to see some overfitting in our learning phase.

Here we will use Keras tools to modify our traning images by flip/rotation/zoom to create new instances.

In [ ]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal", input_shape=(64, 64, 1)),
    layers.RandomRotation(0.25),
    layers.RandomZoom(0.2),
  ]
)

In [ ]:
plt.figure(figsize=(10, 7))
for images, _ in train_ds.take(1):
  for i in range(9):
    augmented_images = data_augmentation(images)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_images[0].numpy())
    plt.axis("off")

In [ ]:
model = Sequential([
  data_augmentation,
  layers.Conv2D(16, 3, padding='same', input_shape=(64, 64, 1), activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.25),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.25),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  #layers.Dropout(0.25),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

We can also try to compile our model with SGD + Momentum. use a learning rate of 0.01 and learning rate decay (15 epochs). Use the usual value for momentum and activate nesterov acceleration:

opt = SGD(learning_rate=0.01, decay=0.01 / 15, momentum=0.9, nesterov=True)


In [ ]:
from tensorflow.keras.optimizers import SGD

opt = SGD(learning_rate=0.01, weight_decay=0.01 / 15, momentum=0.9, nesterov=True)
model.compile(optimizer=opt,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
# train the model
epochs = 15
history = model.fit(
  train_ds,
  validation_data=test_ds,
  epochs=epochs
)

Now evaluate the model.

In [ ]:
predictions = model.predict(X_test, batch_size=128)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, predictions.argmax(axis=1), target_names=labels))

In [ ]:
# plot the training loss and accuracy
plt.figure()
plt.plot(history.history["loss"], label="train_loss")
plt.plot(history.history["val_loss"], label="val_loss")
plt.plot(history.history["accuracy"], label="train_acc")
plt.plot(history.history["val_accuracy"], label="val_acc")
plt.title('Training Loss and Accuracy')
plt.xlabel('Epoch #')
plt.ylabel('Loss/Accuracy')
plt.legend()
plt.show()

Finally label a few images with their prediction

In [ ]:
N = 8
M = 4
indices = np.random.randint(0, y_test.shape[0], size=N*M)

In [ ]:
plt.figure(figsize=(15, 8))
for i in range(N * M):
    ax = plt.subplot(M, N, i + 1)
    plt.imshow(X_test[indices[i]])
    plt.axis('off')
    plt.title('%s' % labels[predictions[indices[i]].argmax()])
plt.show()